# Figure 8. Compare vaccine strains to estimated and observed closest strains to the future

Observed distance to natural H3N2 populations one year into the future for each vaccine strain (green) and the observed (blue) and estimated (orange) closest strains to the future at the corresponding timepoints. Vaccine strains were assigned to the validation or test timepoint closest to the date they were selected by the WHO. The distance to the future of each vaccine strain was calculated from their amino acid sequences and the frequencies and sequences of the corresponding population one year in the future. The estimated closest strain to the future was identified by either the best model in the validation period (mutational load and LBI) or the best model in the test period
(HI antigenic novelty and mutational load).

In [ ]:
validation_tip_attributes_path = snakemake.input.validation_tip_attributes
test_tip_attributes_path = snakemake.input.test_tip_attributes

cTiter_x_ne_star_validation_forecasts_path = snakemake.input.cTiter_x_ne_star_validation_forecasts_path
ne_star_lbi_validation_forecasts_path = snakemake.input.ne_star_lbi_validation_forecasts_path
naive_validation_forecasts_path = snakemake.input.naive_validation_forecasts_path

cTiter_x_ne_star_test_forecasts_path = snakemake.input.cTiter_x_ne_star_test_forecasts_path
ne_star_lbi_test_forecasts_path = snakemake.input.ne_star_lbi_test_forecasts_path
naive_test_forecasts_path = snakemake.input.naive_test_forecasts_path

vaccines_json_path = snakemake.input.vaccines_json_path
titers_path = snakemake.input.titers

output_figure = snakemake.output.figure
output_relative_figure = snakemake.output.relative_figure
output_source_data = snakemake.output.source_data

In [ ]:
"""
validation_tip_attributes_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding/tip_attributes_with_weighted_distances.tsv"
validation_forecasts_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding/forecasts.tsv"

test_tip_attributes_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding_test_tree/tip_attributes_with_weighted_distances.tsv"
test_forecasts_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding_test_tree/forecasts.tsv"

vaccines_json_path = "../config/vaccines_h3n2.json"

output_figure = "../manuscript/figures/vaccine-comparison.pdf"
"""

In [ ]:
from augur.titer_model import TiterCollection
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
from scipy.stats import pearsonr, spearmanr, probplot
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

In [ ]:
register_matplotlib_converters()

In [ ]:
first_validation_timepoint = "2003-10-01"

In [ ]:
sns.set_style("ticks")

In [ ]:
# Display figures at a reasonable default size.
mpl.rcParams['figure.figsize'] = (6, 4)

# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
    
# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 200
mpl.rcParams['figure.dpi'] = 200

# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 18
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

mpl.rc('text', usetex=False)

## Define functions

In [ ]:
def calculate_pairwise_distance(sequence_1, sequence_2):
    return (
        np.frombuffer(sequence_1.encode(), dtype="S1") !=
        np.frombuffer(sequence_2.encode(), dtype="S1")
    ).sum()
    
def calculate_weighted_distance_between_pairs(row):
    distance = calculate_pairwise_distance(
        row["aa_sequence"],
        row["aa_sequence_future"]
    )
    return row["frequency"] * distance

def calculate_weighted_distance_by_group(group_df):
    return group_df.apply(calculate_weighted_distance_between_pairs, axis=1).sum()

## Load tip attributes with sequences

For each timepoint, find distinct sequences that will be used to calculate distances to the future.

In [ ]:
validation_tips_with_sequence = pd.read_csv(
    validation_tip_attributes_path,
    sep="\t",
    parse_dates=["timepoint"],
    usecols=["strain", "timepoint", "frequency", "aa_sequence"]
)

In [ ]:
test_tips_with_sequence = pd.read_csv(
    test_tip_attributes_path,
    sep="\t",
    parse_dates=["timepoint"],
    usecols=["strain", "timepoint", "frequency", "aa_sequence"]
)

In [ ]:
# Retain only validation tips whose timepoints occur prior to the first
# test timepoint. This prevents us from using too many tips from overlapping
# timepoints between validation and test periods.
validation_tips_with_sequence = validation_tips_with_sequence[
    (validation_tips_with_sequence["timepoint"] < test_tips_with_sequence["timepoint"].min()) &
    (validation_tips_with_sequence["timepoint"] >= first_validation_timepoint)
].copy()

In [ ]:
last_validation_timepoint = validation_tips_with_sequence["timepoint"].max()

In [ ]:
last_validation_timepoint

In [ ]:
distinct_validation_tips_with_sequence = validation_tips_with_sequence.groupby(
    ["timepoint", "aa_sequence"]
).first().reset_index()

In [ ]:
distinct_test_tips_with_sequence = test_tips_with_sequence.groupby(
    ["timepoint", "aa_sequence"]
).first().reset_index()

In [ ]:
tips_with_sequence = pd.concat([validation_tips_with_sequence, test_tips_with_sequence])

In [ ]:
distinct_tips_with_sequence = pd.concat([distinct_validation_tips_with_sequence, distinct_test_tips_with_sequence])

## Load vaccine strain data

Load information about vaccine strains including their names, amino acid sequences, and the timepoint in our analysis when they were selected for the vaccine. These latter timepoints constrain the timepoints we consider in the analyses that follow.

In [ ]:
with open(vaccines_json_path, "r") as fh:
    vaccines_json = json.load(fh)

In [ ]:
vaccine_df = pd.DataFrame([
    {
        "strain_type": "vaccine",
        "strain": vaccine,
        "timepoint": vaccine_data["vaccine"]["timepoint"],
        "aa_sequence": vaccine_data["aa_sequence"],
        "abbreviation": vaccine_data["vaccine"]["abbreviation"]
    }
    for vaccine, vaccine_data in vaccines_json["nodes"].items()
])
vaccine_df["timepoint"] = pd.to_datetime(vaccine_df["timepoint"])
vaccine_df["future_timepoint"] = vaccine_df["timepoint"] + pd.DateOffset(months=12)

In [ ]:
# Find all tips with sequences at the future timepoint for each vaccine strain.
tips_for_vaccines_df = vaccine_df.merge(
    tips_with_sequence,
    left_on=["future_timepoint"],
    right_on=["timepoint"],
    suffixes=["", "_future"]
)

In [ ]:
vaccine_distance_to_future = tips_for_vaccines_df.groupby("timepoint").apply(
    calculate_weighted_distance_by_group
).reset_index(name="distance_to_future")

In [ ]:
vaccine_forecasts = vaccine_df.merge(
    vaccine_distance_to_future,
    on="timepoint"
).loc[:, ["strain_type", "strain", "abbreviation", "timepoint", "future_timepoint", "distance_to_future", "aa_sequence"]]

In [ ]:
vaccine_forecasts

In [ ]:
vaccine_strains = vaccine_forecasts["strain"].values

In [ ]:
vaccine_strains

## Load titer data

In [ ]:
titers, titer_strains, titer_sources = TiterCollection.load_from_file(titers_path)

In [ ]:
vaccine_titers = TiterCollection.filter_strains(titers, vaccine_strains)

In [ ]:
titer_collection = TiterCollection(titers)

In [ ]:
titer_collection.normalize_titers()

In [ ]:
len(titer_collection.titers_normalized)

In [ ]:
records = []
for key, log2_titer in titer_collection.titers_normalized.items():
    if key[1][0] in vaccine_strains:
        records.append({
            "reference": key[1][0],
            "virus": key[0],
            "log2_titer": log2_titer
        })

In [ ]:
len(records)

In [ ]:
titers_df = pd.DataFrame(records)

In [ ]:
titers_df["reference"].value_counts()

## Load forecasts from models

### Load forecasts from mutational load and LBI model

In [ ]:
ne_star_lbi_validation_forecasts = pd.read_csv(
    ne_star_lbi_validation_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
)

In [ ]:
ne_star_lbi_validation_forecasts = ne_star_lbi_validation_forecasts.query(
    "timepoint >= '%s'" % first_validation_timepoint
).dropna().copy()

In [ ]:
ne_star_lbi_test_forecasts = pd.read_csv(
    ne_star_lbi_test_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
).dropna()

In [ ]:
ne_star_lbi_forecasts = pd.concat([
    ne_star_lbi_validation_forecasts,
    ne_star_lbi_test_forecasts
])
ne_star_lbi_forecasts = ne_star_lbi_forecasts[
    ne_star_lbi_forecasts["timepoint"].isin(vaccine_forecasts["timepoint"])
].copy()

### Load forecasts from HI antigenic novelty and mutational load model

In [ ]:
cTiter_x_ne_star_validation_forecasts = pd.read_csv(
    cTiter_x_ne_star_validation_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
)

In [ ]:
cTiter_x_ne_star_validation_forecasts = cTiter_x_ne_star_validation_forecasts.query(
    "timepoint >= '%s'" % first_validation_timepoint
).dropna().copy()

In [ ]:
cTiter_x_ne_star_test_forecasts = pd.read_csv(
    cTiter_x_ne_star_test_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
).dropna()

In [ ]:
cTiter_x_ne_star_forecasts = pd.concat([
    cTiter_x_ne_star_validation_forecasts,
    cTiter_x_ne_star_test_forecasts
])
cTiter_x_ne_star_forecasts = cTiter_x_ne_star_forecasts[
    cTiter_x_ne_star_forecasts["timepoint"].isin(vaccine_forecasts["timepoint"])
].copy()

### Load forecasts from naive model

In [ ]:
naive_validation_forecasts = pd.read_csv(
    naive_validation_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
)

In [ ]:
naive_validation_forecasts = naive_validation_forecasts.query(
    "timepoint >= '%s'" % first_validation_timepoint
).dropna().copy()

In [ ]:
naive_test_forecasts = pd.read_csv(
    naive_test_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
).dropna()

In [ ]:
naive_forecasts = pd.concat([
    naive_validation_forecasts,
    naive_test_forecasts
])
naive_forecasts = naive_forecasts[
    naive_forecasts["timepoint"].isin(vaccine_forecasts["timepoint"])
].copy()

## Identify observed closest strains to the future

Use strains from forecast models with annotated weighted distances to the future to find observed closest strains to the future per timepoint with a vaccine.

In [ ]:
combined_forecasts = pd.concat([
    ne_star_lbi_forecasts,
    cTiter_x_ne_star_forecasts
])

In [ ]:
observed_closest_strains = combined_forecasts.sort_values(["timepoint", "weighted_distance_to_future"]).groupby(
    "timepoint"
).first().reset_index().loc[:, ["timepoint", "future_timepoint", "strain", "weighted_distance_to_future"]]
observed_closest_strains["strain_type"] = "observed closest"
observed_closest_strains = observed_closest_strains.rename(
    columns={"weighted_distance_to_future": "distance_to_future"}
)

In [ ]:
observed_closest_strains["relative_distance_to_future"] = (
    observed_closest_strains["distance_to_future"] - vaccine_forecasts["distance_to_future"]
)

In [ ]:
observed_closest_strains

## Identify estimated closest strains to the future by model

Use strains from forecast models with annotated weighted distances to the future to find estimated closest strains to the future per timepoint with a vaccine.

### Identify estimated closest strains to the future by mutational load and LBI

In [ ]:
estimated_closest_strains_by_ne_star_lbi = ne_star_lbi_forecasts.sort_values(
    ["timepoint", "y"]
).groupby(
    "timepoint"
).first().reset_index().loc[:, ["timepoint", "future_timepoint", "strain", "weighted_distance_to_future"]]
estimated_closest_strains_by_ne_star_lbi["strain_type"] = "estimated closest by ne_star-lbi"
estimated_closest_strains_by_ne_star_lbi = estimated_closest_strains_by_ne_star_lbi.rename(
    columns={"weighted_distance_to_future": "distance_to_future"}
)

In [ ]:
estimated_closest_strains_by_ne_star_lbi["relative_distance_to_future"] = (
    estimated_closest_strains_by_ne_star_lbi["distance_to_future"] - vaccine_forecasts["distance_to_future"]
)

In [ ]:
estimated_closest_strains_by_ne_star_lbi

### Identify estimated closest strains to the future by HI antigenic novelty and mutational load

In [ ]:
estimated_closest_strains_by_cTiter_x_ne_star = cTiter_x_ne_star_forecasts.sort_values(
    ["timepoint", "y"]
).groupby(
    "timepoint"
).first().reset_index().loc[:, ["timepoint", "future_timepoint", "strain", "weighted_distance_to_future"]]
estimated_closest_strains_by_cTiter_x_ne_star["strain_type"] = "estimated closest by cTiter_x-ne_star"
estimated_closest_strains_by_cTiter_x_ne_star = estimated_closest_strains_by_cTiter_x_ne_star.rename(
    columns={"weighted_distance_to_future": "distance_to_future"}
)

In [ ]:
estimated_closest_strains_by_cTiter_x_ne_star["relative_distance_to_future"] = (
    estimated_closest_strains_by_cTiter_x_ne_star["distance_to_future"] - vaccine_forecasts["distance_to_future"]
)

In [ ]:
estimated_closest_strains_by_cTiter_x_ne_star

### Identify estimated closest strains to the future by naive model

In [ ]:
estimated_closest_strains_by_naive = naive_forecasts.sort_values(
    ["timepoint", "y"]
).groupby(
    "timepoint"
).first().reset_index().loc[:, ["timepoint", "future_timepoint", "strain", "weighted_distance_to_future"]]
estimated_closest_strains_by_naive["strain_type"] = "estimated closest by naive"
estimated_closest_strains_by_naive = estimated_closest_strains_by_naive.rename(
    columns={"weighted_distance_to_future": "distance_to_future"}
)

In [ ]:
estimated_closest_strains_by_naive["relative_distance_to_future"] = (
    estimated_closest_strains_by_naive["distance_to_future"] - vaccine_forecasts["distance_to_future"]
)

In [ ]:
estimated_closest_strains_by_naive

## Plot distance to the future by timepoint and strain type

Compare distances to the future for selected vaccine strains and the observed and estimated closest strains to the future.

In [ ]:
colors_by_strain_type = {
    "vaccine strain": "#2ca02c",
    "observed best": "#1f77b4",
    "estimated best by ne_star-lbi": "#ff7f0e",
    "estimated best by cTiter_x-ne_star": "#9467bd",
    "estimated best by naive": "#000000",
}

Split data frames into validation and test periods.

In [ ]:
observed_closest_strains_for_validation = observed_closest_strains[
    observed_closest_strains["timepoint"] <= last_validation_timepoint
]
estimated_closest_strains_by_cTiter_x_ne_star_for_validation = estimated_closest_strains_by_cTiter_x_ne_star[
    estimated_closest_strains_by_cTiter_x_ne_star["timepoint"] <= last_validation_timepoint
]
estimated_closest_strains_by_ne_star_lbi_for_validation = estimated_closest_strains_by_ne_star_lbi[
    estimated_closest_strains_by_ne_star_lbi["timepoint"] <= last_validation_timepoint
]
estimated_closest_strains_by_naive_for_validation = estimated_closest_strains_by_naive[
    estimated_closest_strains_by_naive["timepoint"] <= last_validation_timepoint
]
vaccine_forecasts_for_validation = vaccine_forecasts[
    vaccine_forecasts["timepoint"] <= last_validation_timepoint
]

In [ ]:
observed_closest_strains_for_test = observed_closest_strains[
    observed_closest_strains["timepoint"] > last_validation_timepoint
]
estimated_closest_strains_by_cTiter_x_ne_star_for_test = estimated_closest_strains_by_cTiter_x_ne_star[
    estimated_closest_strains_by_cTiter_x_ne_star["timepoint"] > last_validation_timepoint
]
estimated_closest_strains_by_ne_star_lbi_for_test = estimated_closest_strains_by_ne_star_lbi[
    estimated_closest_strains_by_ne_star_lbi["timepoint"] > last_validation_timepoint
]
estimated_closest_strains_by_naive_for_test = estimated_closest_strains_by_naive[
    estimated_closest_strains_by_naive["timepoint"] > last_validation_timepoint
]
vaccine_forecasts_for_test = vaccine_forecasts[
    vaccine_forecasts["timepoint"] > last_validation_timepoint
]

In [ ]:
max_distance_to_future = max(
    observed_closest_strains["distance_to_future"].max(),
    estimated_closest_strains_by_ne_star_lbi["distance_to_future"].max(),
    estimated_closest_strains_by_cTiter_x_ne_star["distance_to_future"].max(),
    estimated_closest_strains_by_naive["distance_to_future"].max(),
    vaccine_forecasts["distance_to_future"].max()
)

In [ ]:
max_distance_to_future

In [ ]:
distance_ticks = np.arange(int(np.ceil(max_distance_to_future)) + 2, step=2)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3.5))
#ax = axes[0]

strain_alpha=0.5

# Plot validation results.
ax.plot(
    estimated_closest_strains_by_naive_for_validation["timepoint"],
    estimated_closest_strains_by_naive_for_validation["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by naive"],
    alpha=strain_alpha,
    label="estimated best by naive"
)

ax.plot(
    estimated_closest_strains_by_cTiter_x_ne_star_for_validation["timepoint"],
    estimated_closest_strains_by_cTiter_x_ne_star_for_validation["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by cTiter_x-ne_star"],
    alpha=strain_alpha,
    label="estimated best by HI + mutational load"
)

ax.plot(
    estimated_closest_strains_by_ne_star_lbi_for_validation["timepoint"],
    estimated_closest_strains_by_ne_star_lbi_for_validation["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by ne_star-lbi"],
    alpha=strain_alpha,
    label="estimated best by mutational load + LBI"
)

ax.plot(
    observed_closest_strains_for_validation["timepoint"],
    observed_closest_strains_for_validation["distance_to_future"],
    "o-",
    color=colors_by_strain_type["observed best"],
    alpha=strain_alpha,
    label="observed best"
)

ax.plot(
    vaccine_forecasts_for_validation["timepoint"],
    vaccine_forecasts_for_validation["distance_to_future"],
    "o-",
    color=colors_by_strain_type["vaccine strain"],
    alpha=strain_alpha,
    label="vaccine strain"
)

# Plot test results.
ax.plot(
    estimated_closest_strains_by_naive_for_test["timepoint"],
    estimated_closest_strains_by_naive_for_test["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by naive"],
    alpha=strain_alpha
)

ax.plot(
    estimated_closest_strains_by_cTiter_x_ne_star_for_test["timepoint"],
    estimated_closest_strains_by_cTiter_x_ne_star_for_test["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by cTiter_x-ne_star"],
    alpha=strain_alpha
)

ax.plot(
    estimated_closest_strains_by_ne_star_lbi_for_test["timepoint"],
    estimated_closest_strains_by_ne_star_lbi_for_test["distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by ne_star-lbi"],
    alpha=strain_alpha
)

ax.plot(
    observed_closest_strains_for_test["timepoint"],
    observed_closest_strains_for_test["distance_to_future"],
    "o-",
    color=colors_by_strain_type["observed best"],
    alpha=strain_alpha
)

ax.plot(
    vaccine_forecasts_for_test["timepoint"],
    vaccine_forecasts_for_test["distance_to_future"],
    "o-",
    color=colors_by_strain_type["vaccine strain"],
    alpha=strain_alpha
)

for index, record in vaccine_forecasts.iterrows():
    ax.text(
        record["timepoint"],
        record["distance_to_future"] + 0.3,
        record["abbreviation"],
        fontsize=10,
        horizontalalignment="center"
    )

ax.axvline(
    last_validation_timepoint + pd.Timedelta(days=35),
    ymax=0.75,
    zorder=-10,
    alpha=0.2,
    color="#000000",
    linestyle="--",
    label="end of validation period"
)

ax.legend(
    loc=(0.01, 0.75),
    frameon=False,
    fontsize=10,
    ncol=2
)

ax.set_yticks(distance_ticks)

ax.set_xlabel("Date")
ax.set_ylabel("Weighted distance\nto the future (AAs)")
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig(output_figure, bbox_inches="tight")

Plot the distance to the future for each model relative to the vaccine strain. This view provides a better sense of how much closer to the future an estimated best strain is than the selected vaccine strain.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3.5))
#ax = axes[0]

strain_alpha=0.5

# Plot validation results.
ax.plot(
    estimated_closest_strains_by_naive_for_validation["timepoint"],
    estimated_closest_strains_by_naive_for_validation["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by naive"],
    alpha=strain_alpha,
    label="estimated best by naive"
)

ax.plot(
    estimated_closest_strains_by_cTiter_x_ne_star_for_validation["timepoint"],
    estimated_closest_strains_by_cTiter_x_ne_star_for_validation["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by cTiter_x-ne_star"],
    alpha=strain_alpha,
    label="estimated best by HI + mutational load"
)

ax.plot(
    estimated_closest_strains_by_ne_star_lbi_for_validation["timepoint"],
    estimated_closest_strains_by_ne_star_lbi_for_validation["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by ne_star-lbi"],
    alpha=strain_alpha,
    label="estimated best by mutational load + LBI"
)

ax.plot(
    observed_closest_strains_for_validation["timepoint"],
    observed_closest_strains_for_validation["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["observed best"],
    alpha=strain_alpha,
    label="observed best"
)

# Plot test results.
ax.plot(
    estimated_closest_strains_by_naive_for_test["timepoint"],
    estimated_closest_strains_by_naive_for_test["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by naive"],
    alpha=strain_alpha
)

ax.plot(
    estimated_closest_strains_by_cTiter_x_ne_star_for_test["timepoint"],
    estimated_closest_strains_by_cTiter_x_ne_star_for_test["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by cTiter_x-ne_star"],
    alpha=strain_alpha
)

ax.plot(
    estimated_closest_strains_by_ne_star_lbi_for_test["timepoint"],
    estimated_closest_strains_by_ne_star_lbi_for_test["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["estimated best by ne_star-lbi"],
    alpha=strain_alpha
)

ax.plot(
    observed_closest_strains_for_test["timepoint"],
    observed_closest_strains_for_test["relative_distance_to_future"],
    "o-",
    color=colors_by_strain_type["observed best"],
    alpha=strain_alpha
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    lw=0.75
)

ax.axvline(
    last_validation_timepoint + pd.Timedelta(days=35),
    ymax=0.8,
    zorder=-10,
    alpha=0.2,
    color="#000000",
    linestyle="--",
    label="end of validation period"
)

ax.legend(
    loc=(0.01, 0.75),
    frameon=False,
    fontsize=10,
    ncol=2
)

#ax.set_yticks(distance_ticks)

ax.set_xlabel("Date")
ax.set_ylabel("Weighted distance\nto the future (AAs)\nrelative to vaccine strain")
ax.set_ylim(-8, 3)

plt.tight_layout()
plt.savefig(output_relative_figure, bbox_inches="tight")

Collect data frames into a single data frame for export as source data.

In [ ]:
source_data = np.around(
    pd.concat([
        vaccine_forecasts,
        observed_closest_strains,
        estimated_closest_strains_by_naive,
        estimated_closest_strains_by_ne_star_lbi,
        estimated_closest_strains_by_cTiter_x_ne_star
    ]).drop(columns=["aa_sequence", "future_timepoint"]),
    2
)

In [ ]:
source_data.to_csv(
    output_source_data,
    header=True,
    index=False,
    na_rep="N/A"
)

In [ ]:
last_validation_timepoint + pd.Timedelta(days=5)

In [ ]:
merged_model_and_vaccine_ne_star_lbi = vaccine_forecasts.merge(
    estimated_closest_strains_by_ne_star_lbi,
    on=["timepoint"],
    suffixes=["_vaccine", "_model"]
)

In [ ]:
merged_model_and_vaccine_ne_star_lbi["vaccine_minus_model"] = (
    merged_model_and_vaccine_ne_star_lbi["distance_to_future_vaccine"] - 
    merged_model_and_vaccine_ne_star_lbi["distance_to_future_model"]
)

In [ ]:
merged_model_and_vaccine_ne_star_lbi["vaccine_minus_model"]

In [ ]:
(merged_model_and_vaccine_ne_star_lbi["vaccine_minus_model"] < 0).sum()

In [ ]:
merged_model_and_vaccine_ne_star_lbi[
    merged_model_and_vaccine_ne_star_lbi["vaccine_minus_model"] < 0
]["vaccine_minus_model"].mean()

In [ ]:
merged_model_and_vaccine_ne_star_lbi["vaccine_minus_model"].mean()

In [ ]:
merged_model_and_vaccine_cTiter_x_ne_star = vaccine_forecasts.merge(
    estimated_closest_strains_by_cTiter_x_ne_star,
    on=["timepoint"],
    suffixes=["_vaccine", "_model"]
)

In [ ]:
merged_model_and_vaccine_cTiter_x_ne_star["vaccine_minus_model"] = (
    merged_model_and_vaccine_cTiter_x_ne_star["distance_to_future_vaccine"] -
    merged_model_and_vaccine_cTiter_x_ne_star["distance_to_future_model"]
)

In [ ]:
merged_model_and_vaccine_cTiter_x_ne_star["vaccine_minus_model"]

In [ ]:
(merged_model_and_vaccine_cTiter_x_ne_star["vaccine_minus_model"] < 0).sum()

In [ ]:
merged_model_and_vaccine_cTiter_x_ne_star[
    merged_model_and_vaccine_cTiter_x_ne_star["vaccine_minus_model"] < 0
]["vaccine_minus_model"]

In [ ]:
merged_model_and_vaccine_cTiter_x_ne_star["vaccine_minus_model"].mean()

In [ ]:
merged_model_and_vaccine_naive = vaccine_forecasts.merge(
    estimated_closest_strains_by_naive,
    on=["timepoint"],
    suffixes=["_vaccine", "_model"]
)

In [ ]:
merged_model_and_vaccine_naive["vaccine_minus_model"] = (
    merged_model_and_vaccine_naive["distance_to_future_vaccine"] -
    merged_model_and_vaccine_naive["distance_to_future_model"]
)

In [ ]:
merged_model_and_vaccine_naive["vaccine_minus_model"]

In [ ]:
(merged_model_and_vaccine_naive["vaccine_minus_model"] < 0).sum()

In [ ]:
merged_model_and_vaccine_naive["vaccine_minus_model"].mean()

## Calculate antigenic distance between vaccine strains and observed/estimated best strains

Determine whether observed and estimated closest strains to the future are antigenically distinct from vaccine strains. First, we look for existing HI measurements between each vaccine strain and the "best" strains. If no measurements exist, we identify the first strain with the closest sequence to each best strain and existing titer measurements as a proxy.

In [ ]:
vaccine_forecasts.head()

In [ ]:
closest_strains = pd.concat([
    observed_closest_strains,
    estimated_closest_strains_by_naive,
    estimated_closest_strains_by_ne_star_lbi,
    estimated_closest_strains_by_cTiter_x_ne_star
])

In [ ]:
closest_strains.head()

In [ ]:
closest_strains_and_vaccines = closest_strains.merge(
    vaccine_forecasts,
    on="timepoint",
    suffixes=["", "_vaccine"]
)

In [ ]:
closest_strains_and_vaccines.head()

In [ ]:
tips_with_sequence.head()

In [ ]:
def find_closest_titers_to_vaccine_strain(titers, tips, vaccine_strain, search_strain):
    """Find titer measurement between the vaccine strain and the closest strain by amino acid sequence
    distance to the search strain.
    """
    # Find titer measurements for the given vaccine strain.
    vaccine_titers = titers.query(f"reference == '{vaccine_strain}'").copy()
    
    # Get tips with titer measurements against the vaccine strain.
    strains_with_vaccine_titers = vaccine_titers["virus"].values
    
    # Check for the search strain in tips with vaccine titers before calculating distances.
    if search_strain in strains_with_vaccine_titers:
        distance = 0
        closest_strain = search_strain
        log2_titer = vaccine_titers.loc[vaccine_titers["virus"] == search_strain, "log2_titer"].mean()
    else:
        # Get all tips with vaccine titers.
        tips_with_vaccine_titers = tips[tips["strain"].isin(strains_with_vaccine_titers)].copy()
        
        # Get the sequence of the search strain.
        search_strain_sequence = tips[tips["strain"] == search_strain]["aa_sequence"].values[0]

        # Calculate pairwise distances between tips with vaccine titers and the search strain.
        tips_with_vaccine_titers["distance_to_search_strain"] = tips_with_vaccine_titers.apply(
            lambda row: calculate_pairwise_distance(row["aa_sequence"], search_strain_sequence),
            axis=1
        )
        
        # Select the strain with the minimum distance to the search strain and its distance.
        distance = tips_with_vaccine_titers["distance_to_search_strain"].min()
        index_of_closest_strain = tips_with_vaccine_titers["distance_to_search_strain"].argmin()
        closest_strain = tips_with_vaccine_titers.iloc[index_of_closest_strain]["strain"]
        
        # Get the log2 titer of the closest strain.
        log2_titer = vaccine_titers.loc[vaccine_titers["virus"] == closest_strain, "log2_titer"].mean()
        
    return closest_strain, distance, log2_titer

In [ ]:
antigenic_distances = []
for index, row in closest_strains_and_vaccines.iterrows():
    closest_strain, distance, log2_titer = find_closest_titers_to_vaccine_strain(
        titers_df,
        tips_with_sequence,
        row["strain_vaccine"],
        row["strain"]
    )
    
    antigenic_distances.append({
        "timepoint": row["timepoint"],
        "vaccine_strain": row["strain_vaccine"],
        "model_strain": row["strain"],
        "model": row["strain_type"],
        "closest_strain": closest_strain,
        "distance_to_closest_strain": distance,
        "relative_distance_to_future": row["relative_distance_to_future"],
        "log2_titer": log2_titer
    })

In [ ]:
len(antigenic_distances)

In [ ]:
antigenic_distances_df = pd.DataFrame(antigenic_distances)

In [ ]:
antigenic_distances_df.head()

At some timepoints, multiple models select the same closest strain to the future. To prevent our analysis of antigenic distances from being biased by these redundant entries, we need to drop duplicates before plotting.

In [ ]:
nonredundant_antigenic_distances_df = antigenic_distances_df.drop(columns="model").drop_duplicates()

In [ ]:
nonredundant_antigenic_distances_df.shape

In [ ]:
nonredundant_antigenic_distances_df["distance_to_closest_strain"].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
sns.regplot(
    nonredundant_antigenic_distances_df["relative_distance_to_future"],
    nonredundant_antigenic_distances_df["log2_titer"],
    ax=ax,
    scatter_kws={"alpha": 0.5}
)

ax.axvline(x=0, color="#cccccc", zorder=-10)
ax.axhline(y=0, color="#cccccc", zorder=-10)

ax.set_xlabel("Weighted distance to the future (AAs)\nrelative to vaccine strain")
ax.set_ylabel("Antigenic distance between\nstrains ($log_{2}$ titer drop)")